In [1]:
# using torch.nn package
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # in_channels, out_channels, kernel_size
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 16 channels & 5x5 image, Affine operation : nn.Linear
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # max pooling (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # flateen all dimensions except batch dimension
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [2]:
# backward function is automatically defined by autograd.

# learnable parametes net.parameters()
params = list(net.parameters())
print(len(params))
print(params[0].size())     # conv1's weights

10
torch.Size([6, 1, 5, 5])


In [3]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0413,  0.0794, -0.0021, -0.0911, -0.0176,  0.0085,  0.0404,  0.0471,
          0.0043,  0.0085]], grad_fn=<AddmmBackward>)


In [4]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [5]:
# loss function
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.4545, grad_fn=<MseLossBackward>)


In [6]:
# backprop
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0003,  0.0149, -0.0023, -0.0212, -0.0054, -0.0011])


In [6]:
# update weights

# simple gradient descent algorithm
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)


# we use torch.optim
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr = 0.01)

# in training loop
optimizer.zero_grad()
ouput = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update weights

### References
- PyTorch.org Tutorials : https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#sphx-glr-beginner-blitz-neural-networks-tutorial-py